# Chapter 7: 챗 애플리케이션 만들기
## OpenAI API 빠른 시작

이 노트북은 [Azure OpenAI 샘플 저장소](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst)에서 가져온 것으로, [Azure OpenAI](notebook-azure-openai.ipynb) 서비스를 활용하는 노트북들이 포함되어 있습니다.

Python OpenAI API는 몇 가지 수정만 하면 Azure OpenAI 모델에서도 사용할 수 있습니다. 두 서비스의 차이점에 대해 더 알고 싶다면 여기에서 확인하세요: [Python으로 OpenAI와 Azure OpenAI 엔드포인트 전환 방법](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)


# 개요  
"대형 언어 모델은 텍스트를 텍스트로 변환하는 함수입니다. 입력된 문자열이 주어지면, 대형 언어 모델은 다음에 올 텍스트를 예측하려고 합니다"(1). 이 '빠른 시작' 노트북에서는 사용자에게 LLM의 주요 개념, AML을 시작하기 위한 핵심 패키지 요구 사항, 프롬프트 설계에 대한 간단한 소개, 그리고 다양한 사용 사례에 대한 짧은 예시들을 안내합니다.


## 목차  

[개요](../../../../07-building-chat-applications/python)  
[OpenAI 서비스 사용 방법](../../../../07-building-chat-applications/python)  
[1. OpenAI 서비스 생성하기](../../../../07-building-chat-applications/python)  
[2. 설치](../../../../07-building-chat-applications/python)    
[3. 자격 증명](../../../../07-building-chat-applications/python)  

[사용 사례](../../../../07-building-chat-applications/python)    
[1. 텍스트 요약](../../../../07-building-chat-applications/python)  
[2. 텍스트 분류](../../../../07-building-chat-applications/python)  
[3. 새로운 제품명 생성](../../../../07-building-chat-applications/python)  
[4. 분류기 미세 조정](../../../../07-building-chat-applications/python)  

[참고 자료](../../../../07-building-chat-applications/python)


### 첫 번째 프롬프트 만들기  
이 짧은 연습에서는 OpenAI 모델에 프롬프트를 제출하여 "요약"이라는 간단한 작업을 수행하는 기본적인 방법을 소개합니다.

**단계**:  
1. 파이썬 환경에 OpenAI 라이브러리를 설치하세요  
2. 표준 헬퍼 라이브러리를 불러오고, 생성한 OpenAI 서비스에 사용할 일반적인 OpenAI 보안 자격 증명을 설정하세요  
3. 작업에 사용할 모델을 선택하세요  
4. 모델에 사용할 간단한 프롬프트를 만드세요  
5. 모델 API에 요청을 제출하세요!


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )


### 3. 적합한 모델 찾기  
GPT-3.5-turbo 또는 GPT-4 모델은 자연어를 이해하고 생성할 수 있습니다.


In [ ]:
# Select the General Purpose curie model for text
model = "gpt-3.5-turbo"

## 4. 프롬프트 설계

"대형 언어 모델의 마법은, 방대한 양의 텍스트에서 예측 오류를 최소화하도록 훈련되면서, 모델이 이러한 예측에 유용한 개념들을 스스로 학습하게 된다는 점입니다. 예를 들어, 모델은 다음과 같은 개념들을 익힙니다"(1):

* 철자법
* 문법의 원리
* 바꿔 말하기
* 질문에 답하기
* 대화 이어가기
* 여러 언어로 글쓰기
* 코딩하기
* 등등

#### 대형 언어 모델을 제어하는 방법  
"대형 언어 모델에 입력되는 것들 중에서, 가장 큰 영향을 미치는 것은 바로 텍스트 프롬프트입니다(1).

대형 언어 모델은 여러 방식으로 프롬프트를 받아 출력을 생성할 수 있습니다:

지시: 모델에게 원하는 것을 직접 말하기
완성: 원하는 것의 시작 부분을 제시해 모델이 이어서 완성하게 하기
시연: 모델에게 원하는 것을 보여주기, 방법은 다음 중 하나:
프롬프트에 몇 가지 예시를 포함하기
수백, 수천 개의 예시를 담은 파인튜닝 학습 데이터셋 사용하기"



#### 프롬프트를 만들 때 기본적으로 따라야 할 세 가지 원칙:

**보여주고 설명하기**. 지시, 예시, 또는 둘을 조합해서 원하는 바를 명확하게 전달하세요. 예를 들어, 목록을 알파벳 순으로 정렬하거나 문단의 감정을 분류하고 싶다면, 모델에게 그 의도를 분명히 보여주세요.

**질 좋은 데이터 제공하기**. 분류기를 만들거나 모델이 특정 패턴을 따르도록 하고 싶다면, 충분한 예시를 제공해야 합니다. 예시를 꼭 검토하세요 — 모델은 기본적인 철자 오류를 알아차리고 답변을 줄 만큼 똑똑하지만, 실수가 의도된 것이라 생각할 수도 있고, 그 결과에 영향을 줄 수 있습니다.

**설정값 확인하기.** temperature와 top_p 설정은 모델이 얼마나 결정적으로 답변을 생성할지 조절합니다. 정답이 하나뿐인 질문을 한다면, 이 값을 낮게 설정하는 것이 좋습니다. 더 다양한 답변을 원한다면, 값을 높게 설정할 수 있습니다. 많은 사람들이 이 설정을 "영리함"이나 "창의성"을 조절하는 것으로 오해하는데, 이것이 가장 흔한 실수입니다.


출처: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## 텍스트 요약  
#### 챌린지  
텍스트의 끝에 'tl;dr:'을 추가하여 내용을 요약하세요. 모델이 별도의 추가 지시 없이 다양한 작업을 수행할 수 있음을 확인할 수 있습니다. tl;dr보다 더 구체적인 프롬프트를 사용해 모델의 행동을 바꾸고 요약 방식을 원하는 대로 조정해 볼 수도 있습니다(3).  

최근 연구에서는 대규모 텍스트 코퍼스에 대한 사전 학습과 특정 작업에 대한 파인튜닝을 결합함으로써 많은 NLP 과제와 벤치마크에서 상당한 성과 향상을 보여주었습니다. 이러한 방법은 보통 아키텍처는 작업에 구애받지 않지만, 여전히 수천에서 수만 개의 예시가 포함된 작업별 파인튜닝 데이터셋이 필요합니다. 반면, 인간은 몇 가지 예시나 간단한 지시만으로도 새로운 언어 작업을 수행할 수 있는데, 이는 현재의 NLP 시스템이 여전히 어려워하는 부분입니다. 본문에서는 언어 모델의 규모를 키우면 작업에 구애받지 않는 소수 예시 기반의 성능이 크게 향상되며, 때로는 기존의 최첨단 파인튜닝 방식과 경쟁할 수 있는 수준에 도달할 수 있음을 보여줍니다.



Tl;dr


# 여러 가지 사용 사례를 위한 연습 문제  
1. 텍스트 요약하기  
2. 텍스트 분류하기  
3. 새로운 제품명 만들기


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 텍스트 분류  
#### 도전 과제  
항목을 추론 시점에 제공된 카테고리로 분류하세요. 아래 예시에서는 분류할 카테고리와 텍스트를 프롬프트에 함께 제공합니다(*playground_reference).

고객 문의: 안녕하세요, 제 노트북 키보드의 키 하나가 최근에 부러져서 교체가 필요합니다:

분류된 카테고리:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 새로운 제품 이름 생성하기
#### 도전 과제
예시 단어들을 활용해 제품 이름을 만들어보세요. 여기서는 이름을 지을 제품에 대한 정보를 프롬프트에 포함시켰습니다. 또한, 원하는 패턴을 보여주기 위해 비슷한 예시도 함께 제공합니다. 더 창의적이고 다양한 답변을 얻기 위해 temperature 값을 높게 설정했습니다.

제품 설명: 집에서 사용하는 밀크셰이크 제조기  
시드 단어: 빠른, 건강한, 컴팩트  
제품 이름: 홈셰이커, 핏셰이커, 퀵셰이크, 셰이크메이커

제품 설명: 어떤 발 크기에도 맞는 신발  
시드 단어: 적응형, 핏, 옴니핏


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# 참고 자료  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio 예제](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [GPT-3를 미세 조정하여 텍스트를 분류하는 모범 사례](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# 추가 도움이 필요하신가요

[OpenAI 상업화 팀](AzureOpenAITeam@microsoft.com)


# 기여자
* Louis Li



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서(원어)가 공식적인 기준임을 유의해 주시기 바랍니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 본 번역 사용으로 인해 발생하는 오해나 오역에 대해 당사는 책임을 지지 않습니다.
